<div align="center">

<a href="https://connectomics.readthedocs.io/en/latest/" target="_blank">
<img width="768", src="https://github.com/zudi-lin/pytorch_connectomics/blob/master/.github/logo_fullname.png?raw=true"></a>


<br>
  <a href="https://colab.research.google.com/drive/1JcdQ9ZCdyQdRtUklBoNxZYC_XI0NG3Uo?usp=sharing"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
<br>

This <a href="https://connectomics.readthedocs.io/en/latest/index.html">PyTorch Connectomics</a> (PyTC) notebook presents step-by-step guidance for visualizing neuron segmentation with the <a href="https://snemi3d.grand-challenge.org/">SENMI3D</a> benchmark. We hope that this notebook will help you better comprehend your segmentation result.<br>Please browse the PyTC <a href="https://connectomics.readthedocs.io/en/latest/">Docs</a> for details, see <a href="https://github.com/zudi-lin/pytorch_connectomics">GitHub</a> or <a href="https://connectomics.readthedocs.io/en/latest/about/team.html">contact us</a> for technical support, and join our <a href="https://pytorchconnectomics.slack.com/join/shared_invite/zt-obufj5d1-v5_NndNS5yog8vhxy4L12w#/shared-invite/email">Slack</a> community for questions and discussions!

</div>

# Set up the Environment
Install dependencies and clone GitHub <a href="https://github.com/zudi-lin/pytorch_connectomics">repository</a>.

**BEFORE YOU START:** Go to `Runtime -> Change runtime type` at the top left menu bar and choose `T4 GPU` instead of `CPU` for hardware accelerator. This will significantly accelerate the inference process. (Optional, but recommended)

In [ ]:
%%capture
!pip install neuroglancer imageio h5py
%env PYTHONPATH=

In [ ]:
%%capture
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-latest-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
%%capture
%%bash
rm Miniconda3-latest-Linux-x86_64.sh
conda create -n py3_torch python=3.8

In [ ]:
%%capture
%%bash
source activate py3_torch
conda install pytorch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1 cudatoolkit=11.3 -c pytorch

In [ ]:
%%bash
source activate py3_torch

### Uncheck the following lines to check PyTorch functionality. If you get "True, 1, Tesla T4", you're good to go ###
# python
# import torch
# print(torch.cuda.is_available())
# print(torch.cuda.device_count())
# print(torch.cuda.get_device_name(0))

git clone https://github.com/zudi-lin/pytorch_connectomics.git
cd pytorch_connectomics
pip install --editable .

In [ ]:
%%capture
%%bash
conda create -n zw python=3.7

In [ ]:
%%capture
%%bash
source activate zw
git clone https://github.com/zudi-lin/waterz.git
cd waterz
conda install --yes --file requirements.txt -c conda-forge
python setup.py install
cd ..
rm -rf waterz

# Get Dataset and Pre-trained Model

In [ ]:
%%capture
%%bash
cd pytorch_connectomics

mkdir datasets
cd datasets
mkdir SNEMI3D
cd SNEMI3D
wget http://rhoana.rc.fas.harvard.edu/dataset/snemi.zip
unzip snemi.zip
mv image/test-input.tif ./
rm -rf ./image
rm -rf ./seg
cd ..
cd ..
mkdir outputs
cd outputs
mkdir SNEMI_SwinUNETR
cd SNEMI_SwinUNETR
wget https://huggingface.co/pytc/Tranformer-based/resolve/main/checkpoint_150000.pth.tar?download=true
mv checkpoint_150000.pth.tar?download=true checkpoint_150000.pth.tar
# gdown https://drive.google.com/uc?id=1FNb_MitONBhJBZA3GyLWne1KxjRD5Veh
cd ..
cd ..

# Begin Inference of Affinity Map
***Please do the following steps before starting inference:***

**1)** Please go to `pytorch_connectomics/configs/SNEMI/SNEMI-Base.yaml` and modify the following configs:

> `SYSTEM.NUM_GPUS=1` (`SYSTEM.NUM_CPUS=2` if you're using CPU for inference);

> `INFERENCE.SAMPLES_PER_BATCH=2` (`=1` if using CPU).

**2)** (Optional) Upload your own trained model to `pytorch_connectomics/outputs/SNEMI_SwinUNETR/`.

**3)** (Optional) Upload test labels to `pytorch_connectomics/datasets/SNEMI3D/` if it is available.

In [ ]:
%%bash
source activate py3_torch
cd pytorch_connectomics

python -u scripts/main.py \
--config-base configs/SNEMI/SNEMI-Base.yaml \
--config-file configs/SNEMI/SNEMI-Affinity-SwinUNETR.yaml --inference \
--checkpoint outputs/SNEMI_SwinUNETR/checkpoint_150000.pth.tar

# Get Segmentation

In [ ]:
%%bash
source activate zw
cd pytorch_connectomics

python
import waterz
import numpy as np
import h5py

f_w = h5py.File('pred.h5', 'w')

f_pred = h5py.File('./outputs/SNEMI_SwinUNETR/test/result_swinunetr.h5', 'r')

# affinities is a [3, z, y, x] numpy array of uint8 if predicted by PyTC
affinities = np.asarray(f_pred['/vol0'], dtype='float32')  # model prediction
f_pred.close()
f_w.create_dataset('aff', data=affinities, dtype='uint8')

affinities = affinities / 255.0
# The affinity values in the model prediction are in the interval [0, 255] and the affinity thresholds provided constraint them
# in the interval [0.05, 0.995] hence we divide it by 255 in order to scale it.

# evaluation: vi/rand
seg_gt = None  # segmentation ground truth. If available, the prediction is evaluated against this ground truth and Rand and VI scores are produced.

aff_thresholds = [0.05, 0.995]
seg_thresholds = [0.1, 0.3, 0.6]

seg = waterz.waterz(affinities,
                    seg_thresholds,
                    merge_function='aff50_his256',
                    aff_threshold=aff_thresholds,
                    gt=seg_gt)

# seg will be an array of shape [3, z, y, x]. Since there are 3 segmentation thresholds, we get a result of shape
# [z, y, x] for each threshold.
seg = np.asarray(seg)

f_w.create_dataset('seg', data=seg, dtype=seg.dtype)
f_w.close()

<img align="left" src="https://connectomics.readthedocs.io/en/latest/_images/snemi_affinity.png" width="768">

# Visualize Results with <a href="https://github.com/google/neuroglancer">Neuroglancer</a>
Below is only a basic demonstration of using Neuroglancer for visualization. For more detailed instructions, please follow this <a href="https://connectomics.readthedocs.io/en/latest/external/neuroglancer.html#">link</a>.

In [ ]:
import neuroglancer
import numpy as np
import imageio
import h5py
import os

os.chdir('./pytorch_connectomics/')


def ngLayer(data, res, oo=[0, 0, 0], tt='segmentation'):
    return neuroglancer.LocalVolume(data,
                                    dimensions=res,
                                    volume_type=tt,
                                    voxel_offset=oo)


viewer = neuroglancer.Viewer()

# SNEMI (# 3d vol dim: z,y,x)
D0 = './datasets/SNEMI3D/'  # image
res0 = neuroglancer.CoordinateSpace(names=['z', 'y', 'x'],
                                    units=['nm', 'nm', 'nm'],
                                    scales=[30, 6, 6])
res1 = neuroglancer.CoordinateSpace(names=['c^', 'z', 'y', 'x'],
                                    units=['', 'nm', 'nm', 'nm'],
                                    scales=[3, 30, 6, 6])

print('load im and gt segmentation')
im = imageio.volread(D0 + 'test-input.tif')
pred_f = h5py.File('pred.h5', 'r')

aff_map = pred_f['/aff']
seg = pred_f['/seg']
seg_1, seg_2, seg_3 = seg[0], seg[1], seg[2]

with viewer.txn() as s:
    s.layers.append(name='im', layer=ngLayer(im, res0, tt='image'))
    s.layers.append(name='seg_0.1', layer=ngLayer(seg_1, res0, tt='segmentation'))
    s.layers.append(name='seg_0.3', layer=ngLayer(seg_2, res0, tt='segmentation'))
    s.layers.append(name='seg_0.6', layer=ngLayer(seg_3, res0, tt='segmentation'))
    s.layers.append(name='aff_map',
                    layer=ngLayer(aff_map,
                                  res1,
                                  oo=[0, 0, 0, 0],
                                  tt='image'),
                    shader="""
        void main() {
        emitRGB(vec3(toNormalized(getDataValue(0)),
        toNormalized(getDataValue(1)),
        toNormalized(getDataValue(2))));
        }
    """)

viewer

<img align="left" src="https://connectomics.readthedocs.io/en/latest/_images/screen_VIEWS.png" width="768">